In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import numpy as np
from tensorflow import keras

In [2]:
vgg_model = tf.keras.applications.vgg16.VGG16()
print(type(vgg_model))

preprocess_input = tf.keras.applications.vgg16.preprocess_input

vgg_model.summary()

<class 'keras.engine.functional.Functional'>
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (Ma

In [3]:
train_data = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

In [4]:
BATCH_SIZE = 32
PIC_SIZE = 224

train = train_data.flow_from_directory("./train", target_size=(PIC_SIZE,PIC_SIZE), batch_size=BATCH_SIZE, class_mode="categorical", color_mode='rgb', shuffle=True)

Found 1800 images belonging to 6 classes.


In [16]:
train.class_indices

{'A': 0, 'B': 1, 'C': 2, 'H': 3, 'L': 4, 'O': 5}

In [17]:
hand_indexs = {v: k for k, v in train.class_indices.items()}

In [18]:
hand_indexs

{0: 'A', 1: 'B', 2: 'C', 3: 'H', 4: 'L', 5: 'O'}

In [19]:
import pickle as pkl

pkl.dump(hand_indexs, open("hand_labels.pkl", "wb"))

In [8]:
model = keras.models.Sequential()
for layer in vgg_model.layers[0:-1]:
    model.add(layer)
    
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)      

In [9]:
for layer in model.layers:
    layer.trainable = False
    
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)      

In [10]:
model.add(tf.keras.layers.Dense(6, activation='softmax'))

In [11]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.CategoricalCrossentropy()

model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])

In [13]:
model.fit(train, epochs=20)

print("Model Trained")

Epoch 1/20
57/57 [==============================] - 151s 3s/step - loss: 0.0015 - acc: 1.0000
Epoch 2/20
57/57 [==============================] - 143s 3s/step - loss: 9.5849e-04 - acc: 1.0000
Epoch 3/20
57/57 [==============================] - 142s 2s/step - loss: 7.5826e-04 - acc: 1.0000
Epoch 4/20
57/57 [==============================] - 147s 3s/step - loss: 6.1409e-04 - acc: 1.0000
Epoch 5/20
57/57 [==============================] - 140s 2s/step - loss: 5.1225e-04 - acc: 1.0000
Epoch 6/20
57/57 [==============================] - 139s 2s/step - loss: 4.3254e-04 - acc: 1.0000
Epoch 7/20
57/57 [==============================] - 144s 3s/step - loss: 3.7272e-04 - acc: 1.0000
Epoch 8/20
57/57 [==============================] - 140s 2s/step - loss: 3.2743e-04 - acc: 1.0000
Epoch 9/20
57/57 [==============================] - 142s 2s/step - loss: 2.8607e-04 - acc: 1.0000
Epoch 10/20
57/57 [==============================] - 138s 2s/step - loss: 2.5272e-04 - acc: 1.0000
Epoch 11/20
57/57 [====

In [15]:
model.save("hand_model.h5")